In [51]:
import multiprocessing as mp
import pickle as pkl
from functools import partial

import numpy as np
import torch
from nff.data.features import graph
from rdkit import Chem
from rdkit.Chem import rdMolAlign
from rdkit.ML.Cluster import Butina
from tqdm import tqdm

In [38]:
def add_properties_to_mol(mol, mol_group):
    # props = ['sars_cov_two_active', 'uniqueconfs', 'energy', 'weights', 'geom_id']
    props = ['sars_cov_two_cl_protease_active', 'uniqueconfs', 'energy', 'weights', 'geom_id']
    for prop in props:
        mol.SetProp(prop, str(mol_group[prop]))

In [57]:

def process_single_molecule(mol_group, threshold=0.8):
    mol_list = mol_group['rd_mols']
    energies = mol_group['energy']

    # Handle single conformer case
    if len(mol_list) < 2:
        unified_mol = Chem.Mol(mol_list[0])
        add_properties_to_mol(unified_mol, mol_group)
        return unified_mol

    # Add properties to all molecules
    for mol in mol_list:
        add_properties_to_mol(mol, mol_group)

    # Calculate RMSD distance matrix
    dists = []
    n_mols = len(mol_list)
    for i in range(n_mols):
        for j in range(n_mols):
            if i <= j:  # Skip diagonal and upper triangle
                continue
            rmsd = rdMolAlign.GetBestRMS(mol_list[i], mol_list[j])
            dists.append(rmsd)

    # Cluster using Butina algorithm
    clusters = Butina.ClusterData(dists, n_mols, threshold, isDistData=True, reordering=True)

    # Process each cluster
    kept_conformers = []  # List to store indices of conformers to keep
    for cluster in clusters:
        if len(cluster) == 0:
            continue

        # Find conformer with lowest energy in the cluster
        cluster_energies = [energies[i].item() for i in cluster]
        min_energy_idx = cluster[np.argmin(cluster_energies)]
        kept_conformers.append(min_energy_idx)

    print(f'Selected {len(kept_conformers)} conformers out of {len(mol_list)} conformers for molecule {mol_group["geom_id"]}')

    # Create new molecule with selected conformers
    if kept_conformers:
        base_mol = Chem.Mol(mol_list[kept_conformers[0]])

        # Add remaining conformers
        for i, conf_idx in enumerate(kept_conformers[1:], 1):
            conf = mol_list[conf_idx].GetConformer()
            positions = conf.GetPositions()

            base_mol.AddConformer(
                Chem.Conformer(base_mol.GetNumAtoms()),
                assignId=True
            )
            new_conf = base_mol.GetConformer(i)

            # Copy atomic positions
            for j, pos in enumerate(positions):
                new_conf.SetAtomPosition(j, pos)

        return base_mol
    return None

In [58]:
def cluster_conformers_parallel(dataset, threshold=0.8, n_processes=128):
    # Create process pool
    with mp.Pool(processes=n_processes) as pool:
        # Process molecules in parallel with progress bar
        process_func = partial(process_single_molecule, threshold=threshold)
        unified_mols = list(tqdm(
            pool.imap(process_func, dataset),
            total=len(dataset),
            desc="Processing molecules"
        ))

    # Remove None values (if any)
    unified_mols = [mol for mol in unified_mols if mol is not None]

    return unified_mols

In [ ]:
threshold = 0.8
n_processes = 128

for path in ['test.pth.tar', 'train.pth.tar', 'val.pth.tar']:
    print(f"Processing {path}")
    dataset = torch.load(path)
    dataset = graph.make_rd_mols(dataset)

    unified_mols = cluster_conformers_parallel(dataset, threshold, n_processes)

    pkl.dump(unified_mols, open(f'cov2_{path.split(".")[0]}.pkl', 'wb'))

Processing test.pth.tar


100%|██████████| 162/162 [00:06<00:00, 23.44it/s]

Converted 162 of 162 species (100.00%)
No elements are missing from xyz2mol



Processing molecules:   0%|          | 0/162 [00:00<?, ?it/s]

Selected 1 conformers out of 2 conformers for molecule 104932004
Selected 2 conformers out of 4 conformers for molecule 131019989
Selected 4 conformers out of 11 conformers for molecule 131038306
Selected 5 conformers out of 8 conformers for molecule 130827383
Selected 5 conformers out of 7 conformers for molecule 131178926
Selected 1 conformers out of 2 conformers for molecule 131854732
Selected 9 conformers out of 11 conformers for molecule 130840695
Selected 3 conformers out of 3 conformers for molecule 131383491
Selected 14 conformers out of 21 conformers for molecule 136478832
Selected 7 conformers out of 14 conformers for molecule 133199359
Selected 7 conformers out of 16 conformers for molecule 133329552
Selected 2 conformers out of 7 conformers for molecule 130892778Selected 4 conformers out of 6 conformers for molecule 130871478

Selected 6 conformers out of 18 conformers for molecule 132424581
Selected 44 conformers out of 61 conformers for molecule 131390144
Selected 3 confo

Processing molecules:   1%|          | 2/162 [00:39<52:57, 19.86s/it]

Selected 21 conformers out of 74 conformers for molecule 131672782
Selected 43 conformers out of 73 conformers for molecule 136643359
Selected 8 conformers out of 45 conformers for molecule 131613029
Selected 11 conformers out of 20 conformers for molecule 131651735
Selected 30 conformers out of 62 conformers for molecule 118231610
Selected 27 conformers out of 200 conformers for molecule 132456729
Selected 6 conformers out of 8 conformers for molecule 130782767
Selected 31 conformers out of 71 conformers for molecule 131543661
Selected 49 conformers out of 113 conformers for molecule 132053341
Selected 3 conformers out of 8 conformers for molecule 130976384
Selected 38 conformers out of 108 conformers for molecule 131785202
Selected 29 conformers out of 85 conformers for molecule 136779271


Processing molecules:   4%|▎         | 6/162 [1:01:34<26:41:01, 615.78s/it]Process ForkPoolWorker-337:
Process ForkPoolWorker-316:
Process ForkPoolWorker-325:
Process ForkPoolWorker-340:
Process ForkPoolWorker-334:
Process ForkPoolWorker-344:
Process ForkPoolWorker-322:
Process ForkPoolWorker-308:
Process ForkPoolWorker-341:
Process ForkPoolWorker-324:
Process ForkPoolWorker-304:
Process ForkPoolWorker-307:
Process ForkPoolWorker-319:
Process ForkPoolWorker-335:
Process ForkPoolWorker-318:
Process ForkPoolWorker-321:
Process ForkPoolWorker-288:
Process ForkPoolWorker-343:
Process ForkPoolWorker-317:
Process ForkPoolWorker-313:
Process ForkPoolWorker-330:
Process ForkPoolWorker-310:
Process ForkPoolWorker-306:
Process ForkPoolWorker-294:
Process ForkPoolWorker-315:
Process ForkPoolWorker-303:
Process ForkPoolWorker-267:
Process ForkPoolWorker-268:
Process ForkPoolWorker-269:
Process ForkPoolWorker-305:
Process ForkPoolWorker-338:
Process ForkPoolWorker-326:
Process ForkPoolWorker-279:
P